In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("venmo.csv")

In [39]:
df.loc[df['note'].str.contains('drugs', na=False)]

,note
1159,🌯🌯🌯 and drugs
5190,"thx 4 tha drugs, homie"
7967,Not drugs
14628,drugs
17041,Thanks for the drugs ma
...,...
7058829,thx for the drugs
7059718,Not drugs
7064162,Ground up drugs
7073149,drugs and hugs


In [40]:
df.loc[df['note'].str.contains('meth', na=False)]

,note
744,Happy birthday baby sis! Go buy yourself some candy or something.👑🤡🕯🎉🎊🎊🎈🎈🎈
4246,Just a little something to get you started
5325,Eat something that isn't protein or workout supplements.
6042,:venmo_dollar: it said something about exceeding my limit when trying to send 5700
6081,"Cowboys and country 🎵, take 20 or something like that."
...,...
7073165,Buy yourself a latte or something bc you are so talented and really I owe you much more than this 💕💕💕💕💕
7073175,"The square root of 69 is 8 something, right? Cuz I’ve been trynna work it out aaahhahh..."
7074943,Something
7076144,To buy stuff to clean your FRAT! 🍻😉. \nJust a little something for my GODSON! \nLove Ya Bud!


In [42]:
df.loc[df['note'].str.contains('weed', na=False)]

,note
4281,"Trash Removal, Lawn mowed, weed whack"
4959,For the weed and crack and hoes
12671,+1 weed
14524,weed
18210,weed
...,...
7022928,Half of Doug’s soul wrapped in seaweed and rice
7037091,"Weed money (crap, not weed money)"
7040159,White girls and weed
7064977,for ya weeds


In [43]:
df.loc[df['note'].str.contains('marijuana', na=False)]

,note
41913,thc marijuana ganja weed dope OG on cloud 9 higher than a kite cartridge
137012,3 grams of marijuana
235888,marijuana
450590,"Dear Mr.Ikeda\nThank you so much for your generosity, I could not be more grateful. I will never forget the time that you paid for me at Shokudo on @Quinn-Arakaki birthday, it has touched my heart and inspired me to pay for others in time of need. The other day @Christie-Lee-20 didn’t have enough money for lunch so I thought to myself “hmm I should pay for christie like that time Blake paid for me” so I did I spared her the few bucks she needed to pay for her meal. At that point and on I kind of made a little quote “what would Blake do” and when I think of that I think of generous acts I can do for people in need, I also created a hashtag just for fun #wwbd. You know we’ve known each other for about 4-5 years and I’ve learned a lot about you, like how you snore like a cow when you sleep, and how you like to drive without music so that you can hear your baby purr as you cruise down the H1 and I’ve learned all this by spending quality time with my boi (you). From that moment we met in high school our friendship has blossomed like a beautiful bud on a marijuana plant, although you don’t smoke the devils lettuce I’m glad you accept me for the pot head I am and our friendship can live on till the end of time, because that’s how long our friendship is gonna last. We’re gonna be together till the end. I love you 😘 Ps: I wrote you a little poem. Roses are red, violets are blue, I smoke pot and I wish you did too, you paid for me at Shokudo, now take this money for a poke bowl, I don’t care what you do with the money, maybe go out and treat your hunny, maybe get yourself a couple bunnies, but know this, I will forever be grateful, and never be hateful, I luh you buubuu and I hope you do to. Love Jace Tamayei❤️"
452536,"Dear Ms.Arakaki \n\nSo today is your birthday so I thought to give you a little something to treat yourself with, you’re welcome. We met somewhere around freshmen year, and I had to be good friends with you in order to get @Christie-Lee-20 so that’s where our friendship began and just like @Balakayikeda and I, from then on our friendship blossomed like a beautiful bud on a marijuana plant. As we hung out more I began to learn more and more about you like how you chew with your mouth open or how picky you are when people are on the aux but it’s okay I’ve learned to love it all, especially your singing, the first time I heard your song I fell in love with it and so did daboiz, I remember when I got that live performance from you and I literally got goosebumps because as your vocal chords vibrated in your throat and shot out sound waves traveling through the air hitting my ears flowing down my ear canal into my ear drums a voice of an angel resonated in my skull and I will never forget that moment and everything about it. The air, the smell, my car, your voice, the little hairs on my skin raising as you sang. I hope you had the best birthday ever! \nPs: I wrote you a rap\nHit the beat🎵🎶🎵🎶\nI’m chillin with my home girl Quinn \nWhen I’m with her all we do is win\nYuh my home girl I never bone\nBut home girl we can talk on the phone\nAll night long \nas we hit a fat bong\nIt’s your birthday Q\nAnd we all scream YEEEEEEW\n\nI luh you Quinn❤️\nLove Jace Tamayei"
1091142,Not marijuana
1341637,One marijuana
1444526,"Whoopie, whoopie, whoopie, marijuana!"
1542114,Not marijuana
1623290,Not marijuana


In [47]:
df.loc[df['note'].str.contains('alcohol', na=False)]

,note
1539,Blame it on the alcohol
5171,Last call for alcohollllllllllllllllll for #lastcallforcarvajallllllllllllllll
7616,Canned alcohol
15185,Don’t waste it all on alcohol
17432,Get alcohol for our pregame
...,...
7043550,alcohol
7045230,Room alcohol and small snacks! Last charge wohoo!
7060099,Groceries and alcohol
7068447,"Tax, tip, and alcohol - El T"


In [3]:
df.loc[df['note'].str.contains('🔌', na=False)]

,note
34,💡🔌🔌🔌🔋
59,💦🔌🔥
119,🔌
182,July ⚡🔌💡💸
300,💦🔌🔥
...,...
7076389,🍪🔌
7076405,💡🔌
7076474,⚡🔌💡💸
7076486,"⚡🔌💡💸 saw it on the counter this morning, being..."
